A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor



# Import any additional modules and start coding below
df = pd.read_csv('rental_info.csv')

# to calculate rental length days, change datatype to datetime format
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])

#calculate date diff
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

# adding dummy variables using special features column
df['deleted_scenes'] = np.where(df['special_features'].str.contains("Deleted Scenes"),1,0)
df['behind_the_scenes'] = np.where(df['special_features'].str.contains("Behind the Scenes"),1,0)

# Separate target data from df and drop redundant columns
X = df.drop(['special_features','rental_date','return_date','rental_length_days'], axis=1)
y = df['rental_length_days']

In [3]:
# split data into train and test data
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size= 0.2, random_state= 9)

# Performing feature selection
lasso = Lasso(alpha=0.01, random_state=9)  
lasso.fit(X_train, y_train)

lasso_coef = lasso.coef_
#print(lasso_coef)
 
X_train_selected = X_train.loc[:, lasso_coef > 0]
X_test_selected = X_test.loc[:, lasso_coef > 0]

# LinearRegression
model = LinearRegression()
model.fit(X_train_selected,y_train)
y_pred = model.predict(X_test_selected)
mse = mean_squared_error(y_test, y_pred)
print(mse)

# RandomForestRegression
rf = RandomForestRegressor(random_state=9)
rf.fit(X_train_selected, y_train)
rf_pred = rf.predict(X_test_selected)

rf_mse = mean_squared_error(y_test, rf_pred)
print("rf_mse", rf_mse)



4.846638768993796
rf_mse 2.3608479570462744
